<a href="https://colab.research.google.com/github/21cs076/Real-Time-Landslide-Detection-and-Buried-Vehicle-Identification-using-YOLOv8/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('/content/best736 (2).pt')

# Perform inference on an image from Google Drive
results = model('/content/Wp.jpg')

# Iterate through the results and display each
for result in results:
    result.show()

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4MptKsSsONT0fnQsIDc2")
project = rf.workspace("deslizamentos").project("landslide-detection-augmentation")
version = project.version(12)
dataset = version.download("yolov8")


In [ ]:
from ultralytics import YOLO
from google.colab import files

# Load a pre-trained YOLOv8 model
model = YOLO("/content/best736.pt")

# Train the model on your Landslide dataset
model.train(
    data="/content/Landslide-Detection-Augmentation-12/data.yaml",  # Path to your data.yaml file
    epochs=100,  # Number of epochs (adjust based on your needs)
    batch=16,   # Batch size (adjust based on GPU memory), (16), 32, 64, 128
    imgsz=640,  # Image size for training ((640 is a good choice), 736)
    project="runs/train",  # Directory to save training results
    name="landslide_detection",  # Name for the training run
    exist_ok=True  # Overwrite existing results if any
)

# Download the saved model
files.download("runs/train/landslide_detection/weights/best.pt")

In [ ]:
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("runs/train/landslide_detection/weights/best.pt")

# Evaluate the model using the val() method
metrics = model.val(data="/content/Landslide-Detection-Augmentation-12/data.yaml", split='test')

# Print evaluation metrics
print(f"mAP50: {metrics.box.map50}") # mAP refers to the Average Precision at a single IoU threshold, often 50%.
print(f"mAP75: {metrics.box.map75}") # mAP at IoU threshold of 75% (mAP75)

In [ ]:
from google.colab import files
uploaded = files.upload()

import os
os.listdir('/content')

In [ ]:
!pip install onnx

import torch
import onnx
import numpy as np

# Assuming 'model' is already defined and loaded with your trained model
onnx_model_path = "/content/Landslide-Detection-Augmentation-12/best.onnx"
torch.onnx.export(
    model,
    torch.randn(1, 3, 640, 640),  # Dummy input for the model
    onnx_model_path,
    opset_version=11,
    input_names=["input"],
    output_names=["output"]
)

# Download the saved model
files.download("/content/Landslide-Detection-Augmentation-12/best.onnx")

In [ ]:
# Clone YOLOv8 repository
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics